# 环境安装

    本项目来源于和鲸社区，使用转载需要标注来源
    作者: 和鲸社区
    来源: 和鲸社区
    使用https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple 清华园镜像

In [1]:
!ls

BuildDataset.py     ModelMerge2.py  combined_processed_data_arabic.json
DataProcess.py	    ModelTest1.py   train.py
DomainEvaluator.py  ModelTest2.py   阿拉伯语微调实践.ipynb
ModelMerge1.py	    Welcome.ipynb


# 环境导入

In [2]:
import os
import logging
# 设置 HF_ENDPOINT 环境变量
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
logging.basicConfig(level=logging.INFO)
print(os.environ.get('HF_ENDPOINT'))

https://hf-mirror.com


In [3]:
import torch  # PyTorch深度学习框架
from datasets import load_dataset, Dataset  # Hugging Face的数据集加载工具
from transformers import (  # Hugging Face的转换器库
    AutoModelForCausalLM,  # 自回归语言模型（用于生成文本）
    AutoTokenizer,  # 自动分词器
    AutoModelForMaskedLM,  # 掩码语言模型
    get_linear_schedule_with_warmup,  # 学习率预热调度器
    DataCollatorForLanguageModeling  # 用于MLM的数据整理器
)
from peft import LoraConfig, get_peft_model, TaskType  # 参数高效微调工具
from torch.utils.data import DataLoader, random_split  # 数据加载相关工具
from torch.optim import AdamW  # Adam优化器的变体
from tqdm import tqdm  # 进度条工具
import os
import random
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import logging
from datetime import datetime
import json
import random
import re
import os
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict, Counter
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
import math
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba
import pythainlp
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import word_tokenize as thai_tokenize
from pythainlp import pos_tag as thai_pos_tag
from typing import List, Dict, Set
import langdetect
from langdetect.lang_detect_exception import LangDetectException

/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:datasets:PyTorch version 2.7.0 available.


# 模型评估与优化 （确定评估指标）
- 困惑度(Perplexity)
    - 评估模型在验证集上的表现
    - 监控模型是否过拟合
- 领域适应性评估
    - 术语覆盖率：# term_coverage是术语覆盖率，计算response中包含的领域术语数量占总术语数量的比例
    - 术语密度：# term_density是术语密度，计算response中包含的领域术语数量占总token数量的比例
    - 响应质量：# response_quality是回复质量，计算response与prompt的相似度

In [4]:
class DomainEvaluator:
    def __init__(self, tokenizer, device):
        self.tokenizer = tokenizer
        self.device = device
        # 加载sentence transformer用于计算文本相似度
        self.sentence_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        self.sentence_model.to(device)
        
        # 加载领域术语
        with open("/gemini/data-1/domain_terms_arabic.txt", "r", encoding="utf-8") as f:
            self.domain_terms = [line.strip() for line in f if line.strip()]
        
    def calculate_domain_perplexity(self, model, eval_dataloader):
        """计算领域数据的困惑度"""
        model.eval()
        total_loss = 0
        total_tokens = 0
        
        with torch.no_grad():
            for batch in eval_dataloader:
                batch = {k: v.to(self.device) for k, v in batch.items()}
                outputs = model(**batch)
                total_loss += outputs.loss.item() * batch["input_ids"].size(0)
                total_tokens += batch["input_ids"].ne(self.tokenizer.pad_token_id).sum().item()
        
        return torch.exp(torch.tensor(total_loss / total_tokens))
    
    def evaluate_domain_adaptation(self, model, texts, lang=None):
        """评估生成文本的领域适应性"""
        model.eval()
        metrics = {
            "term_coverage": [],
            "term_density": [],
            "response_quality": []
        }
        
        with torch.no_grad():
            for text in texts:
                # 构建提示
                if lang:
                    prompt = f"请用{lang}语言回答以下问题：\n{text}"
                else:
                    prompt = text
                
                # 生成回复
                inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(self.device)
                outputs = model.generate(
                    **inputs,
                    max_length=800,
                    num_return_sequences=1,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.95
                )
                response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                
                # 计算评估指标
                term_count = sum(1 for term in self.domain_terms if term.lower() in response.lower())
                term_coverage = term_count / len(self.domain_terms) if self.domain_terms else 0
                metrics["term_coverage"].append(term_coverage)
                
                term_density = term_count / len(response) if response else 0
                metrics["term_density"].append(term_density)
                
                # 计算回复质量
                response_embedding = self.sentence_model.encode([response])
                prompt_embedding = self.sentence_model.encode([text])
                similarity = cosine_similarity(response_embedding, prompt_embedding)[0][0]
                metrics["response_quality"].append(similarity)
        
        return {
            "avg_term_coverage": np.mean(metrics["term_coverage"]),
            "avg_term_density": np.mean(metrics["term_density"]),
            "avg_response_quality": np.mean(metrics["response_quality"])
        }

# 数据处理
数据处理流程
1. 数据清洗
2. 去除HTML标签和特殊字符
3. 规范化文本格式
4. 过滤过长或过短的文本
5. 数据转换
6. 构建输入输出对
7. 添加任务相关的指令提示

In [5]:
import json
import gzip
from pathlib import Path
import re
from tqdm import tqdm
import random
import os
import pandas as pd

In [6]:
def clean_text(text):
    """清理文本内容"""
    if not text:
        return text
    
    # 正则表达式是一种文本匹配模式,下面详细解释每一步:
    
    # 1. 处理连续换行符
    # re.sub()函数用于替换文本,接受3个参数:
    # - 第1个参数 r'\n+' 表示:
    #   \n 代表换行符
    #   + 表示匹配1个或多个连续的换行符
    # - 第2个参数 '\n' 表示用单个换行符替换
    # - 第3个参数是要处理的文本
    # strip()去除文本首尾的空格
    text = re.sub(r'\n+', '\n', text.strip())
    
    # 2. 处理连续空格
    # r'\s+' 表示:
    # \s 代表任意空白字符(空格、制表符等)
    # + 表示匹配1个或多个连续的空白字符
    # 用单个空格替换所有连续的空白字符
    text = re.sub(r'\s+', ' ', text)
    
    # 3. 移除HTML标签，正则表达式 r'<[^>]+>'
    # 1) r'' 表示这是一个原始字符串,不会对反斜杠\进行转义处理
    # 2) < 就是匹配HTML标签的开始符号 <
    # 3) [^>] 是一个字符集:
    #    - [] 表示匹配其中的任意一个字符
    #    - ^ 在[]内表示"非",即取反
    #    - 所以[^>]表示匹配任何不是>的字符
    # 4) + 表示"一个或多个",即重复前面的[^>]一次或多次
    # 5) > 就是匹配HTML标签的结束符号 >
    # 
    # 举例说明:
    # 原文本: "这是<p>一个段落</p>"
    # - <p> 会被匹配,因为它符合模式:<加上任意非>字符(这里是p)再加上>
    # - </p> 也会被匹配,因为它符合模式:<加上任意非>字符(这里是/p)再加上>
    # 
    # re.sub()会把所有匹配到的内容替换为空字符串'',所以最后变成:
    # "这是一个段落"
    text = re.sub(r'<[^>]+>', '', text)
    return text.strip()


In [7]:
def process_item(item):
    """处理单条数据，统一不同数据集的格式"""
    try:
        # 检查数据格式并提取必要字段
        if isinstance(item, dict):
            # 常见格式：包含title和content的字典
            if 'title' in item and 'content' in item:
                title = clean_text(item['title'])
                content = clean_text(item['content'])
            # 其他可能的格式
            elif 'text' in item:
                # 如果只有text字段，尝试从文本中提取标题
                text = clean_text(item['text'])
                lines = text.split('\n', 1)
                if len(lines) > 1:
                    title, content = lines
                else:
                    title = "文章"
                    content = text
            else:
                return None
        else:
            return None
            
        # 检查文本长度
        if len(content) < 50 or len(content) > 10000:
            return None
            
        # 构建统一的训练格式
        conversation = {
            "instruction": f"请生成一段带有阿拉伯专业术语的文本。\n\n标题: {title}",
            "input": "",
            "output": content,
            "category": item.get('labels', {}).get('pjwk_cates', "general")
        }
        
        return conversation
    except Exception as e:
        print(f"处理数据时出错: {e}")
        return None

In [8]:
def process_file(input_path, sample_ratio=0.1):
    """处理单个文件并随机抽样"""
    processed_data = []
    
    try:
        # 读取gzip文件
        with gzip.open(input_path, 'rt', encoding='utf-8') as f:
            # 首先读取所有行
            lines = f.readlines()
            
            # 随机抽样
            # 确保sample_size至少为1,避免抽样失败
            sample_size = max(1, int(len(lines) * sample_ratio))
            # 从lines列表中随机抽取sample_size条数据
            sampled_lines = random.sample(lines, sample_size)
            
            # 处理抽样的数据
            for line in tqdm(sampled_lines, desc=f"处理文件 {Path(input_path).name}"):
                try:
                    item = json.loads(line.strip())
                    processed_item = process_item(item)
                    if processed_item:
                        processed_data.append(processed_item)
                except json.JSONDecodeError:
                    continue
                except Exception as e:
                    print(f"处理数据时出错: {e}")
                    continue
    except Exception as e:
        print(f"处理文件 {input_path} 时出错: {e}")
    
    return processed_data

In [9]:
def process_all_datasets(base_dir, output_path, sample_ratio=0.25):
    """处理所有数据集并合并结果"""
    all_processed_data = []
    
    # 递归查找所有.jsonl.gz文件
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.jsonl.gz'):
                input_path = os.path.join(root, file)
                print(f"\n处理文件: {input_path}")
                
                # 处理单个文件
                file_data = process_file(input_path, sample_ratio)
                all_processed_data.extend(file_data)
                
                print(f"从 {file} 中提取了 {len(file_data)} 条数据")
    
    # 保存所有处理后的数据
    output_path = Path(output_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(all_processed_data, f, ensure_ascii=False, indent=2)
    
    print(f"\n处理完成！")
    print(f"总共处理了 {len(all_processed_data)} 条数据")
    print(f"数据已保存至: {output_path}")
    
    # 输出数据集统计信息
    categories = {}
    for item in all_processed_data:
        cat = item['category']
        if isinstance(cat, dict):
            cat = str(cat)
        categories[cat] = categories.get(cat, 0) + 1
    
    print("\n数据集类别分布:")
    for cat, count in categories.items():
        print(f"{cat}: {count} 条")

In [10]:
def main():
    # 设置随机种子以确保可重复性
    random.seed(42)
    
    # 输入输出路径
    base_dir = "/gemini/data-1/"  # 原始数据集目录
    output_file = "/gemini/code/combined_processed_data_arabic.json"  # 输出文件路径

    # 处理所有数据集
    process_all_datasets(base_dir, output_file, sample_ratio=0.25)

    # 显示样例数据
    with open(output_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        print("\n处理后的数据样例:")
        print(json.dumps(data[0], ensure_ascii=False, indent=2))

In [12]:
if __name__ == "__main__":
    main()


处理文件: /gemini/data-1/阿拉伯part-677f75d865d8-001143.jsonl.gz


处理文件 阿拉伯part-677f75d865d8-001143.jsonl.gz: 100%|██████████| 569725/569725 [00:08<00:00, 69539.19it/s]


从 阿拉伯part-677f75d865d8-001143.jsonl.gz 中提取了 20976 条数据

处理完成！
总共处理了 20976 条数据
数据已保存至: /gemini/code/combined_processed_data_arabic.json

数据集类别分布:
{'level1': ['professional_field'], 'level2': ['technology']}: 10707 条
{'level1': ['professional_field'], 'level2': ['finance']}: 2881 条
{'level1': ['professional_field'], 'level2': ['law']}: 124 条
{'level1': ['professional_field'], 'level2': ['academic']}: 6182 条
{'level1': ['professional_field'], 'level2': ['patent']}: 619 条
{'level1': ['professional_field'], 'level2': ['institutions']}: 371 条
{'level1': ['professional_field'], 'level2': ['education']}: 92 条

处理后的数据样例:
{
  "instruction": "请生成一段带有阿拉伯专业术语的文本。\n\n标题: آخر الأخبار: ifor",
  "input": "",
  "output": "الأخبار التقنية أعلنت اليوم شركة إنفور، الرائدة في مجال توريد تطبيقات المؤسسية المتخصصة والقائمة على السحابة، عن مشاركتها واستعراضها لعروض حزمة CloudSuite…",
  "category": {
    "level1": [
      "professional_field"
    ],
    "level2": [
      "technology"
    ]
  }
}


# 提取专业术语：如“操作系统”、“高血压”
注意事项：先撇去阿拉伯语语种的特性影响，如音节，再进行分词，这样的分词效果更好

In [11]:
def extract_domain_terms_by_language(text: str, lang: str) -> List[str]:
    """使用语言专属工具提取领域术语"""
    terms = []
    if lang == 'ar':
        try:
            # 使用pyarabic进行基本处理
            text = araby.strip_tashkeel(text)  # 移除变音符号
            words = araby.tokenize(text)
            # 提取可能的术语（长度大于3的词）
            terms = [w for w in words if len(w) > 3 and not any(c.isdigit() for c in w)]
        except Exception as e:
            print(f"阿拉伯语处理失败: {e}")
    
    return terms

In [12]:
def extract_domain_terms(domain_texts: List[str], general_texts: List[str] = None, top_n: int = 2000) -> List[str]:
    """使用改进的多语言分词方法提取领域术语"""
    print("开始提取领域术语...")
    
    # 按语言分组处理文本
    lang_texts = defaultdict(list)
    for text in domain_texts:
        lang = detect_language(text)
        if lang != 'unknown':
            lang_texts[lang].append(text)
    
    print("\n文本语言分布:")
    for lang, texts in lang_texts.items():
        print(f"- {lang}: {len(texts)} 条")
    
    # 分语言处理并提取术语
    all_terms = []
    for lang, texts in lang_texts.items():
        print(f"\n处理{lang}语言文本...")
        
        # 使用语言专属工具提取术语
        lang_terms = set()
        for text in tqdm(texts, desc=f"处理{lang}语言文本"):
            terms = extract_domain_terms_by_language(text, lang)
            lang_terms.update(terms)
        
        # 过滤和排序术语
        lang_terms = list(lang_terms)
        # 计算每个术语在文本中出现的频率
        # 1. 遍历每个文本
        # 2. 对每个文本重新提取术语
        # 3. 使用Counter统计所有术语的频率
        term_freq = Counter(t for text in texts for t in extract_domain_terms_by_language(text, lang))
        
        # 对术语列表进行排序:
        # 1. 首要排序依据是术语出现频率(term_freq[x])
        # 2. 次要排序依据是术语长度(len(x))
        # reverse=True表示按降序排列,即频率高的和长度长的排在前面
        lang_terms.sort(key=lambda x: (term_freq[x], len(x)), reverse=True)
        
        # 根据语言数量平均分配术语数量配额
        # 1. top_n是总的期望术语数量
        # 2. len(lang_texts)是语言种类数
        # 3. 对每种语言,只取配额内的高频长术语
        # //是整除运算符,用于计算每种语言分配的术语数量配额
        # 例如:如果top_n=2000,有4种语言,则每种语言分配2000//4=500个术语
        # 这里的:是切片操作符,表示从列表开头取到指定位置
        # //是整除运算符,例如10//3=3
        # 所以lang_terms[:top_n // len(lang_texts)]表示:
        # 1. 先计算top_n除以语言数量的整除结果n
        # 2. 然后从lang_terms列表中取前n个元素
        selected_terms = lang_terms[:top_n // len(lang_texts)]
        all_terms.extend(selected_terms)
        
        print(f"{lang}语言提取了 {len(selected_terms)} 个术语")
        if selected_terms:
            print(f"{lang}语言术语示例:")
            for term in selected_terms[:5]:
                print(f"  - {term}")
    
    return all_terms

# 数据集构建
- 将原始数据转换为指令微调格式
- 训练集和验证集划分
- 数据格式标准化

In [13]:
def load_and_prepare_data(json_path, tokenizer, max_length=512, val_ratio=0.1):
    """加载并预处理微调数据（标准LLM微调流程）"""
    with open(json_path, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)
    
    # 记录每个样本中"### Response:\n"的字符位置
    formatted_texts = []
    response_char_positions = []
    for item in raw_data:
        prompt = item['prompt']
        output = item['output']
        formatted_text = f"### Instruction:\n{prompt}\n\n### Response:\n{output}"
        formatted_texts.append(formatted_text)
        # 找到"### Response:\n"的字符位置
        response_start_char = formatted_text.find("### Response:\n") + len("### Response:\n")
        response_char_positions.append(response_start_char)
    
    # 分词处理
    tokenized_data = tokenizer(
        formatted_texts,
        max_length=max_length,
        truncation=True,
        padding="max_length",  # 修改：使用max_length填充
        add_special_tokens=True,
        return_offsets_mapping=True  # 需要字符到token的映射
    )
    
    # 准备标签
    labels = []
    for i in range(len(tokenized_data["input_ids"])):
        input_ids = tokenized_data["input_ids"][i]
        offsets = tokenized_data["offset_mapping"][i]
        
        # 通过字符位置找到response起始的token索引
        response_start_token = None
        for token_idx, (char_start, char_end) in enumerate(offsets):
            if char_start >= response_char_positions[i]:
                response_start_token = token_idx
                break
        
        # 如果未找到（例如被截断），则设为整个序列
        if response_start_token is None:
            response_start_token = len(input_ids)
        
        # 创建label：只保留response部分的token_id
        label = [-100] * len(input_ids)
        label[response_start_token:] = input_ids[response_start_token:]
        labels.append(label)
    
    # 移除offset_mapping（不再需要）
    tokenized_data.pop("offset_mapping")
    
    # 创建数据集
    dataset = Dataset.from_dict({
        "input_ids": tokenized_data["input_ids"],
        "attention_mask": tokenized_data["attention_mask"],
        "labels": labels
    })
    
    # 划分训练验证集
    split_dataset = dataset.train_test_split(test_size=val_ratio, seed=42)
    
    # 转换为PyTorch张量格式
    def set_format(ds):
        ds.set_format(type='torch', columns=["input_ids", "attention_mask", "labels"])
        return ds
    
    train_dataset = set_format(split_dataset["train"])
    val_dataset = set_format(split_dataset["test"])
    
    # 修正后的collate_fn - 确保所有序列长度一致
    def collate_fn(batch):
        # 将batch中的每个tensor转为列表以便处理
        input_ids = [item["input_ids"].tolist() for item in batch]
        attention_mask = [item["attention_mask"].tolist() for item in batch]
        labels = [item["labels"].tolist() for item in batch]
        
        # 找出最大长度
        max_len = max(len(ids) for ids in input_ids)
        
        # 进行右侧填充
        for i in range(len(batch)):
            pad_len = max_len - len(input_ids[i])
            if pad_len > 0:
                input_ids[i] += [tokenizer.pad_token_id] * pad_len
                attention_mask[i] += [0] * pad_len
                labels[i] += [-100] * pad_len  # 使用-100填充标签
        
        # 转换回tensor
        result = {
            "input_ids": torch.tensor(input_ids),
            "attention_mask": torch.tensor(attention_mask),
            "labels": torch.tensor(labels)
        }
        
        return result
    
    # 创建DataLoader
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=4,
        shuffle=True,
        collate_fn=collate_fn
    )
    val_dataloader = DataLoader(
        val_dataset,
        batch_size=4,
        collate_fn=collate_fn
    )
    
    return train_dataloader, val_dataloader

# 编码：数据处理成适合MLM的格式

- 功能描述
    该函数用于加载和预处理用于大语言模型微调的数据。它将原始JSON格式的训练数据转换为适合模型训练的格式,并创建训练和验证数据加载器。

- 参数说明
    - json_path (str): 输入JSON文件的路径
    - tokenizer: 用于文本分词的tokenizer对象
    - max_length (int): 序列的最大长度,默认为512
    - val_ratio (float): 验证集比例,默认为0.1
-处理流程
    1. 数据加载与格式化

        - 从JSON文件加载原始数据
        - 将每个样本格式化为"### Instruction:\n{prompt}\n\n### Response:\n{output}"的形式
        - 记录每个样本中"### Response:\n"的字符位置
    2. 分词处理

        - 使用tokenizer对文本进行分词
        - 设置最大长度并进行截断
        - 使用max_length进行填充
        - 获取字符到token的映射信息
    3. 标签准备

        - 通过字符位置找到response起始的token索引
        - 创建标签序列:
            - response之前的token标记为-100(忽略)
            - response部分保留原token_id
    4. 数据集创建与划分

        - 使用Huggingface的Dataset类创建数据集
        - 按照设定比例划分训练集和验证集
        - 将数据格式转换为PyTorch张量
    5. DataLoader创建

        - 实现collate_fn确保batch中序列长度一致
        - 创建训练和验证数据加载器
        - 设置batch_size为4
- 返回值
返回一个元组,包含:

  - train_dataloader: 训练数据加载器
  - val_dataloader: 验证数据加载器

In [14]:
import json
from datasets import Dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

def load_and_prepare_data(json_path, tokenizer, max_length=512, val_ratio=0.1):
    """加载并预处理微调数据（标准LLM微调流程）"""
    with open(json_path, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)
    
    # 记录每个样本中"### Response:\n"的字符位置
    formatted_texts = []
    response_char_positions = []
    for item in raw_data:
        prompt = item['prompt']
        output = item['output']
        formatted_text = f"### Instruction:\n{prompt}\n\n### Response:\n{output}"
        formatted_texts.append(formatted_text)
        # 找到"### Response:\n"的字符位置
        response_start_char = formatted_text.find("### Response:\n") + len("### Response:\n")
        response_char_positions.append(response_start_char)
    
    # 分词处理
    tokenized_data = tokenizer(
        formatted_texts,
        max_length=max_length,
        truncation=True,
        padding="max_length",  # 修改：使用max_length填充
        add_special_tokens=True,
        return_offsets_mapping=True  # 需要字符到token的映射
    )
    
    # 准备标签
    labels = []
    for i in range(len(tokenized_data["input_ids"])):
        input_ids = tokenized_data["input_ids"][i]
        offsets = tokenized_data["offset_mapping"][i]
        
        # 通过字符位置找到response起始的token索引
        response_start_token = None
        for token_idx, (char_start, char_end) in enumerate(offsets):
            if char_start >= response_char_positions[i]:
                response_start_token = token_idx
                break
        
        # 如果未找到（例如被截断），则设为整个序列
        if response_start_token is None:
            response_start_token = len(input_ids)
        
        # 创建label：只保留response部分的token_id
        label = [-100] * len(input_ids)
        label[response_start_token:] = input_ids[response_start_token:]
        labels.append(label)
    
    # 移除offset_mapping（不再需要）
    tokenized_data.pop("offset_mapping")
    
    # 创建数据集
    dataset = Dataset.from_dict({
        "input_ids": tokenized_data["input_ids"],
        "attention_mask": tokenized_data["attention_mask"],
        "labels": labels
    })
    
    # 划分训练验证集
    split_dataset = dataset.train_test_split(test_size=val_ratio, seed=42)
    
    # 转换为PyTorch张量格式
    def set_format(ds):
        ds.set_format(type='torch', columns=["input_ids", "attention_mask", "labels"])
        return ds
    
    train_dataset = set_format(split_dataset["train"])
    val_dataset = set_format(split_dataset["test"])
    
    # 修正后的collate_fn - 确保所有序列长度一致
    def collate_fn(batch):
        # 将batch中的每个tensor转为列表以便处理
        input_ids = [item["input_ids"].tolist() for item in batch]
        attention_mask = [item["attention_mask"].tolist() for item in batch]
        labels = [item["labels"].tolist() for item in batch]
        
        # 找出最大长度
        max_len = max(len(ids) for ids in input_ids)
        
        # 进行右侧填充
        for i in range(len(batch)):
            pad_len = max_len - len(input_ids[i])
            if pad_len > 0:
                input_ids[i] += [tokenizer.pad_token_id] * pad_len
                attention_mask[i] += [0] * pad_len
                labels[i] += [-100] * pad_len  # 使用-100填充标签
        
        # 转换回tensor
        result = {
            "input_ids": torch.tensor(input_ids),
            "attention_mask": torch.tensor(attention_mask),
            "labels": torch.tensor(labels)
        }
        
        return result
    
    # 创建DataLoader
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=4,
        shuffle=True,
        collate_fn=collate_fn
    )
    val_dataloader = DataLoader(
        val_dataset,
        batch_size=4,
        collate_fn=collate_fn
    )
    
    return train_dataloader, val_dataloader

# 模型加载与训练——deepseek1.5B&开始训练
1. 模型准备
    - 加载DeepSeek基础模型
    - 配置tokenizer
    - 设置LoRA参数
2. 训练配置
    - 学习率设置
    - 批次大小选择
    - 训练轮次确定
    - 优化器选择
3. 训练过程
    - 梯度更新
    - 学习率调度
    - 模型保存
    - 训练监控

In [15]:
# 设置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'training_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)


In [16]:
# 设置随机种子以确保结果可重现
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [21]:
# class DomainEvaluator:
#     def __init__(self, tokenizer, device):
#         self.tokenizer = tokenizer
#         self.device = device
#         # 加载sentence transformer用于计算文本相似度
#         self.sentence_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
#         self.sentence_model.to(device)
        
#         # 加载领域术语
#         with open("/home/mw/input/Arabic314891489/domain_terms_arabic.txt", "r", encoding="utf-8") as f:
#             self.domain_terms = [line.strip() for line in f if line.strip()]
        
#     def calculate_domain_perplexity(self, model, eval_dataloader):
#         """计算领域数据的困惑度"""
#         model.eval()
#         total_loss = 0
#         total_tokens = 0
        
#         with torch.no_grad():
#             for batch in eval_dataloader:
#                 batch = {k: v.to(self.device) for k, v in batch.items()}
#                 outputs = model(**batch)
#                 total_loss += outputs.loss.item() * batch["input_ids"].size(0)
#                 total_tokens += batch["input_ids"].ne(self.tokenizer.pad_token_id).sum().item()
        
#         return torch.exp(torch.tensor(total_loss / total_tokens))
    
#     def evaluate_domain_adaptation(self, model, texts, lang=None):
#         """评估生成文本的领域适应性"""
#         model.eval()
#         metrics = {
#             "term_coverage": [],
#             "term_density": [],
#             "response_quality": []
#         }
        
#         with torch.no_grad():
#             for text in texts:
#                 # 构建提示
#                 if lang:
#                     prompt = f"请用{lang}语言回答以下问题：\n{text}"
#                 else:
#                     prompt = text
                
#                 # 生成回复
#                 inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(self.device)
#                 outputs = model.generate(
#                     **inputs,
#                     max_length=800,
#                     num_return_sequences=1,
#                     do_sample=True,
#                     temperature=0.7,
#                     top_p=0.95
#                 )
#                 response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                
#                 # 计算评估指标
#                 term_count = sum(1 for term in self.domain_terms if term.lower() in response.lower())
#                 term_coverage = term_count / len(self.domain_terms) if self.domain_terms else 0
#                 metrics["term_coverage"].append(term_coverage)
                
#                 term_density = term_count / len(response) if response else 0
#                 metrics["term_density"].append(term_density)
                
#                 # 计算回复质量
#                 response_embedding = self.sentence_model.encode([response])
#                 prompt_embedding = self.sentence_model.encode([text])
#                 similarity = cosine_similarity(response_embedding, prompt_embedding)[0][0]
#                 metrics["response_quality"].append(similarity)
        
#         return {
#             "avg_term_coverage": np.mean(metrics["term_coverage"]),
#             "avg_term_density": np.mean(metrics["term_density"]),
#             "avg_response_quality": np.mean(metrics["response_quality"])
#         }

# import json
# from datasets import Dataset
# from transformers import AutoTokenizer
# from torch.utils.data import DataLoader

# def load_and_prepare_data(json_path, tokenizer, max_length=512, val_ratio=0.1):
#     """加载并预处理微调数据（标准LLM微调流程）"""
#     with open(json_path, 'r', encoding='utf-8') as f:
#         raw_data = json.load(f)
    
#     # 记录每个样本中"### Response:\n"的字符位置
#     formatted_texts = []
#     response_char_positions = []
#     for item in raw_data:
#         prompt = item['prompt']
#         output = item['output']
#         formatted_text = f"### Instruction:\n{prompt}\n\n### Response:\n{output}"
#         formatted_texts.append(formatted_text)
#         # 找到"### Response:\n"的字符位置
#         response_start_char = formatted_text.find("### Response:\n") + len("### Response:\n")
#         response_char_positions.append(response_start_char)
    
#     # 分词处理
#     tokenized_data = tokenizer(
#         formatted_texts,
#         max_length=max_length,
#         truncation=True,
#         padding="max_length",  # 修改：使用max_length填充
#         add_special_tokens=True,
#         return_offsets_mapping=True  # 需要字符到token的映射
#     )
    
#     # 准备标签
#     labels = []
#     for i in range(len(tokenized_data["input_ids"])):
#         input_ids = tokenized_data["input_ids"][i]
#         offsets = tokenized_data["offset_mapping"][i]
        
#         # 通过字符位置找到response起始的token索引
#         response_start_token = None
#         for token_idx, (char_start, char_end) in enumerate(offsets):
#             if char_start >= response_char_positions[i]:
#                 response_start_token = token_idx
#                 break
        
#         # 如果未找到（例如被截断），则设为整个序列
#         if response_start_token is None:
#             response_start_token = len(input_ids)
        
#         # 创建label：只保留response部分的token_id
#         label = [-100] * len(input_ids)
#         label[response_start_token:] = input_ids[response_start_token:]
#         labels.append(label)
    
#     # 移除offset_mapping（不再需要）
#     tokenized_data.pop("offset_mapping")
    
#     # 创建数据集
#     dataset = Dataset.from_dict({
#         "input_ids": tokenized_data["input_ids"],
#         "attention_mask": tokenized_data["attention_mask"],
#         "labels": labels
#     })
    
#     # 划分训练验证集
#     split_dataset = dataset.train_test_split(test_size=val_ratio, seed=42)
    
#     # 转换为PyTorch张量格式
#     def set_format(ds):
#         ds.set_format(type='torch', columns=["input_ids", "attention_mask", "labels"])
#         return ds
    
#     train_dataset = set_format(split_dataset["train"])
#     val_dataset = set_format(split_dataset["test"])
    
#     # 修正后的collate_fn - 确保所有序列长度一致
#     def collate_fn(batch):
#         # 将batch中的每个tensor转为列表以便处理
#         input_ids = [item["input_ids"].tolist() for item in batch]
#         attention_mask = [item["attention_mask"].tolist() for item in batch]
#         labels = [item["labels"].tolist() for item in batch]
        
#         # 找出最大长度
#         max_len = max(len(ids) for ids in input_ids)
        
#         # 进行右侧填充
#         for i in range(len(batch)):
#             pad_len = max_len - len(input_ids[i])
#             if pad_len > 0:
#                 input_ids[i] += [tokenizer.pad_token_id] * pad_len
#                 attention_mask[i] += [0] * pad_len
#                 labels[i] += [-100] * pad_len  # 使用-100填充标签
        
#         # 转换回tensor
#         result = {
#             "input_ids": torch.tensor(input_ids),
#             "attention_mask": torch.tensor(attention_mask),
#             "labels": torch.tensor(labels)
#         }
        
#         return result
    
#     # 创建DataLoader
#     train_dataloader = DataLoader(
#         train_dataset,
#         batch_size=4,
#         shuffle=True,
#         collate_fn=collate_fn
#     )
#     val_dataloader = DataLoader(
#         val_dataset,
#         batch_size=4,
#         collate_fn=collate_fn
#     )
    
#     return train_dataloader, val_dataloader

In [17]:
def find_optimal_lora_config(model, train_dataloader, val_dataloader, device, evaluator):
    """搜索最优的LoRA配置，使用小数据集快速搜索"""
    configs = [
        {"r": 16, "alpha": 64},
        {"r": 8, "alpha": 32},
        {"r": 4, "alpha": 16}
    ]
    
    best_perplexity = float('inf')
    best_config = None
    
    # 从训练集和验证集中各取100条数据创建小数据集
    small_train_data = []
    small_val_data = []
    
    # 收集小训练集
    train_iter = iter(train_dataloader)
    for _ in range(min(25, len(train_dataloader))):  # 25个batch * 4 = 100条数据
        try:
            batch = next(train_iter)
            small_train_data.append(batch)
        except StopIteration:
            break
    
    # 收集小验证集
    val_iter = iter(val_dataloader)
    for _ in range(min(25, len(val_dataloader))):
        try:
            batch = next(val_iter)
            small_val_data.append(batch)
        except StopIteration:
            break
    
    logger.info(f"创建了小数据集用于配置搜索：训练集 {len(small_train_data)} 批次，验证集 {len(small_val_data)} 批次")
    
    for config in configs:
        logger.info(f"\n测试LoRA配置: r={config['r']}, alpha={config['alpha']}")
        
        # 配置LoRA
        lora_config = LoraConfig(
            r=config['r'],
            lora_alpha=config['alpha'],
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
            lora_dropout=0.05,
            bias="none",
            task_type=TaskType.CAUSAL_LM
        )
        
        # 创建PEFT模型
        peft_model = get_peft_model(model, lora_config)
        
        # 快速训练和评估
        optimizer = AdamW(peft_model.parameters(), lr=2e-4)
        peft_model.train()
        
        # 在小训练集上训练
        for _ in range(2):  # 只训练2个epoch
            for batch in small_train_data:
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = peft_model(**batch)
                loss = outputs.loss
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
        
        # 在小验证集上评估
        peft_model.eval()
        total_loss = 0
        total_tokens = 0
        with torch.no_grad():
            for batch in small_val_data:
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = peft_model(**batch)
                total_loss += outputs.loss.item() * batch["input_ids"].size(0)
                total_tokens += batch["input_ids"].ne(evaluator.tokenizer.pad_token_id).sum().item()
        # 计算困惑度，原理是计算验证集的损失除以验证集的总token数量，然后取指数，目的是衡量模型的困惑度
        perplexity = torch.exp(torch.tensor(total_loss / total_tokens))
        logger.info(f"配置性能 - 困惑度: {perplexity:.2f}")
        
        if perplexity < best_perplexity:
            best_perplexity = perplexity
            best_config = config
            logger.info(f"找到新的最佳配置！")
    
    return best_config

In [18]:
def convert_metrics_to_json_serializable(metrics):
    """将指标转换为JSON可序列化的格式"""
    if isinstance(metrics, dict):
        return {k: convert_metrics_to_json_serializable(v) for k, v in metrics.items()}
    elif isinstance(metrics, list):
        return [convert_metrics_to_json_serializable(v) for v in metrics]
    elif isinstance(metrics, (torch.Tensor, np.ndarray)):
        return metrics.item() if metrics.size == 1 else metrics.tolist()
    elif isinstance(metrics, (int, float, str, bool)):
        return metrics
    elif metrics is None:
        return None
    else:
        return str(metrics)


In [16]:
def main():
    # 设置随机种子
    set_seed(42)
    
    # 检查CUDA
    assert torch.cuda.is_available(), "需要CUDA支持"
    device = torch.device("cuda:0")
    
    print(os.environ.get('HF_ENDPOINT'))
    print(os.environ.get('HF_ENDPOINT'))
    print(os.environ.get('HF_ENDPOINT'))

    # 加载模型和分词器
    model_path = "/gemini/pretrain/DeepSeek-R1-Distill-Qwen-1.5B"
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        trust_remote_code=True,
        torch_dtype=torch.float16,
    ).to(device)
    
    tokenizer = AutoTokenizer.from_pretrained(
        model_path,
        trust_remote_code=True,
    
    )
    tokenizer.pad_token = tokenizer.eos_token
    
    train_dataloader, val_dataloader = load_and_prepare_data("/gemini/data-1/lora_training_data_arabic.json", tokenizer)
    

    # 初始化评估器
    evaluator = DomainEvaluator(tokenizer, device)
    
    # # 寻找最优LoRA配置
    # logger.info("开始寻找最优LoRA配置...")
    # best_config = find_optimal_lora_config(model, train_dataloader, val_dataloader, device, evaluator)
    # logger.info(f"找到最优LoRA配置: r={best_config['r']}, alpha={best_config['alpha']}")
    
    # # 使用最优配置创建LoRA模型
    # lora_config = LoraConfig(
    #     r=best_config['r'],
    #     lora_alpha=best_config['alpha'],
    #     target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    #     lora_dropout=0.1,
    #     bias="none",
    #     task_type=TaskType.CAUSAL_LM
    # )
    
    lora_config = LoraConfig(
        r=4,
        lora_alpha=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.CAUSAL_LM
    )

    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
    
    # 训练配置
    optimizer = AdamW(model.parameters(), lr=5e-4)
    num_epochs = 3
    num_training_steps = len(train_dataloader) * num_epochs
    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=200,
        num_training_steps=num_training_steps
    )
    
    # 从domain_terms_arabic.txt构建评估提示
    with open("/gemini/data-1/domain_terms_arabic.txt", "r", encoding="utf-8") as f:
        domain_terms = [line.strip() for line in f if line.strip()]
    
    # 构建基于领域术语的评估提示
    unlabeled_eval_prompts = {
        "ar": [
            f"请解释阿拉伯语中'{term}'这个术语的含义和用法。" for term in random.sample(domain_terms, 5)
        ] + [
            f"请用阿拉伯语写一段话，包含以下术语：{', '.join(random.sample(domain_terms, 3))}",
            f"在技术领域中，'{random.choice(domain_terms)}'和'{random.choice(domain_terms)}'这两个术语有什么联系？",
            f"请用阿拉伯语描述'{random.choice(domain_terms)}'在现代技术发展中的应用。",
            f"请生成一段带有阿拉伯专业术语的文本",
            f"请生成一段带有阿拉伯专业术语的文本",
            f"请生成一段带有阿拉伯专业术语的文本",
            f"请生成一段带有阿拉伯专业术语的文本",
            f"请生成一段带有阿拉伯专业术语的文本",
            f"请生成一段带有阿拉伯专业术语的文本",
            f"请生成一段带有阿拉伯专业术语的文本",
            f"请生成一段带有阿拉伯专业术语的文本"
        ]
    }
    
    # 训练循环
    best_metrics = {
        "val_perplexity": float('inf'),
        "domain_adaptation": 0
    }
    metrics_log = []
    eval_steps = 100  # 每300步评估一次
    best_model_path = "/gemini/Arabic314891489/deepseek-lora-best"
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
        print(len(train_dataloader))
        for step, batch in enumerate(progress_bar):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            total_loss += loss.item()
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            
            progress_bar.set_postfix({"loss": loss.item()})
            
            # 评估
            if step % eval_steps == 0 and step > 0:
                model.eval()
                
                # 1. 计算验证集困惑度
                val_perplexity = evaluator.calculate_domain_perplexity(model, val_dataloader)
                
                # 2. 评估领域适应性
                domain_metrics = {}
                for lang, prompts in unlabeled_eval_prompts.items():
                    metrics = evaluator.evaluate_domain_adaptation(model, prompts, lang)
                    domain_metrics[lang] = metrics
                
                # 3. 计算综合指标
                avg_domain_score = np.mean([
                    m["avg_term_coverage"] * 0.4 +
                    m["avg_term_density"] * 0.3 +
                    m["avg_response_quality"] * 0.3
                    for m in domain_metrics.values()
                ])
                
                # 记录当前学习率
                current_lr = optimizer.param_groups[0]["lr"]
                
                metrics = {
                    "epoch": epoch + 1,
                    "step": step,
                    "val_perplexity": val_perplexity.item() if isinstance(val_perplexity, torch.Tensor) else float(val_perplexity),
                    "domain_adaptation_score": float(avg_domain_score),
                    "learning_rate": float(current_lr),
                    "domain_metrics": convert_metrics_to_json_serializable(domain_metrics)
                }
                metrics_log.append(metrics)
                
                logger.info(f"\n验证集困惑度: {val_perplexity:.4f}")
                logger.info(f"领域适应性得分: {avg_domain_score:.4f}")
                logger.info(f"当前学习率: {current_lr:.6f}")
                
                # 4. 保存最佳模型
                combined_score = avg_domain_score/val_perplexity
                if combined_score > best_metrics["domain_adaptation"]/best_metrics["val_perplexity"]:
                    best_metrics["val_perplexity"] = float(val_perplexity)
                    best_metrics["domain_adaptation"] = float(avg_domain_score)
                    model.save_pretrained(best_model_path)
                    logger.info(f"保存新的最佳模型！困惑度={val_perplexity:.4f}, 领域得分={avg_domain_score:.4f}")
                
                model.train()

        # 每个epoch结束保存检查点
        checkpoint_path = f"/gemini/Arabic314891489/deepseek-lora-checkpoint-{epoch+1}"
        try:
            model.save_pretrained(checkpoint_path)
            logger.info(f"已保存Epoch {epoch+1}检查点")
        except Exception as e:
            logger.error(f"保存检查点时出错: {e}")
        
        avg_loss = total_loss / len(train_dataloader)
        logger.info(f"Epoch {epoch+1} 平均损失: {avg_loss:.4f}")
    
    # 保存最终模型和训练指标
    try:
        model.save_pretrained("/gemini/Arabic314891489/deepseek-lora-final")
        with open('training_metrics.json', 'w', encoding='utf-8') as f:
            json.dump(metrics_log, f, ensure_ascii=False, indent=2)
        logger.info("训练完成！已保存最终模型和训练指标")
    except Exception as e:
        logger.error(f"保存最终结果时出错: {e}")

In [19]:
19. 主程序入口
if __name__ == "__main__":
    # 打印环境信息
    logger.info(f"PyTorch version: {torch.__version__}")
    logger.info(f"CUDA available: {torch.cuda.is_available()}")
    logger.info(f"CUDA version: {torch.version.cuda}")
    logger.info(f"GPU count: {torch.cuda.device_count()}")
    logger.info(f"Current GPU: {torch.cuda.current_device()}")
    logger.info(f"GPU name: {torch.cuda.get_device_name(0)}")
    
    main()

# 模型合并

In [19]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType
import json
import logging
from datetime import datetime
import numpy as np
from pathlib import Path

# 设置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'merge_and_save_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [20]:
def convert_metrics_to_json_serializable(metrics):
    """将指标转换为JSON可序列化的格式"""
    if isinstance(metrics, dict):
        return {k: convert_metrics_to_json_serializable(v) for k, v in metrics.items()}
    elif isinstance(metrics, list):
        return [convert_metrics_to_json_serializable(v) for v in metrics]
    elif isinstance(metrics, (torch.Tensor, np.ndarray)):
        return metrics.item() if metrics.size == 1 else metrics.tolist()
    elif isinstance(metrics, (int, float, str, bool)):
        return metrics
    elif metrics is None:
        return None
    else:
        return str(metrics)

In [21]:
def save_metrics_and_merge_model():
    """保存训练指标并合并模型"""
    try:
        # 1. 保存训练指标（如果有）
        if 'metrics_log' in globals():
            logger.info("正在保存训练指标...")
            metrics_log_serializable = [convert_metrics_to_json_serializable(m) for m in metrics_log]
            with open('training_metrics.json', 'w', encoding='utf-8') as f:
                json.dump(metrics_log_serializable, f, ensure_ascii=False, indent=2)
            logger.info("训练指标已保存到 training_metrics.json")
        
        # 2. 加载和合并模型
        logger.info("开始加载模型...")
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # 加载基础模型
        base_model_path = "/gemini/pretrain/DeepSeek-R1-Distill-Qwen-1.5B"
        logger.info(f"加载基础模型: {base_model_path}")
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_path,
            trust_remote_code=True,
            torch_dtype=torch.float16
        ).to(device)
        
        # 加载tokenizer
        tokenizer = AutoTokenizer.from_pretrained(
            base_model_path,
            trust_remote_code=True
        )
        
        # 配置LoRA
        lora_config = LoraConfig(
            r=4,
            lora_alpha=16,
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
            lora_dropout=0.1,
            bias="none",
            task_type=TaskType.CAUSAL_LM
        )
        
        # 创建PEFT模型
        logger.info("应用LoRA配置...")
        model = get_peft_model(base_model, lora_config)
        
        # 加载最佳检查点
        best_model_path = "/gemini/pretrain2/deepseek-lora-best"
        logger.info(f"加载最佳检查点: {best_model_path}")
        model.load_adapter(best_model_path, adapter_name="arabic_adapter")
        
        # 合并权重
        logger.info("合并模型权重...")
        merged_model = model.merge_and_unload()
        
        # 保存合并后的模型
        merged_model_path = "/gemini/Arabic314891489/deepseek-merged"
        logger.info(f"保存合并后的模型到: {merged_model_path}")
        merged_model.save_pretrained(merged_model_path)
        
        # 保存tokenizer
        logger.info("保存tokenizer...")
        tokenizer.save_pretrained(merged_model_path)
        
        logger.info("所有操作完成！")
        
        # 返回模型和tokenizer以供后续使用
        return merged_model, tokenizer
        
    except Exception as e:
        logger.error(f"处理过程中出错: {e}")
        raise

In [22]:
def test_merged_model(model, tokenizer, test_prompts=None):
    """测试合并后的模型"""
    if test_prompts is None:
        # 使用domain_terms.txt中的术语构建测试提示
        try:
            with open("/gemini/data-1/domain_terms_arabic.txt", "r", encoding="utf-8") as f:
                domain_terms = [line.strip() for line in f if line.strip()]
            
            # 随机选择一些术语构建测试提示
            import random
            selected_terms = random.sample(domain_terms, min(5, len(domain_terms)))
            test_prompts = [
                f"请解释阿拉伯语中'{term}'这个术语的含义。" for term in selected_terms
            ]
        except Exception as e:
            logger.warning(f"无法加载domain_terms.txt，使用默认测试提示: {e}")
            test_prompts = [
                f"请用阿拉伯语写一段话，包含以下术语：{', '.join(random.sample(domain_terms, 3))}",
                f"在技术领域中，'{random.choice(domain_terms)}'和'{random.choice(domain_terms)}'这两个术语有什么联系？",
                f"请用阿拉伯语描述'{random.choice(domain_terms)}'在现代技术发展中的应用。",
                f"请生成一段带有阿拉伯专业术语的文本",
                f"请生成一段带有阿拉伯专业术语的文本",
                f"请生成一段带有阿拉伯专业术语的文本",
                f"请生成一段带有阿拉伯专业术语的文本",
                f"请生成一段带有阿拉伯专业术语的文本",
                f"请生成一段带有阿拉伯专业术语的文本",
                f"请生成一段带有阿拉伯专业术语的文本",
                f"请生成一段带有阿拉伯专业术语的文本"
            ]
    
    logger.info("\n开始测试合并后的模型...")
    device = next(model.parameters()).device
    
    for prompt in test_prompts:
        logger.info(f"\n测试提示: {prompt}")
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=512,
                num_return_sequences=1,
                temperature=0.7,
                do_sample=True,
                top_p=0.95,
                repetition_penalty=1.1
            )
        
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        logger.info(f"模型回答: {response}\n")
        logger.info("-" * 50)

In [23]:
if __name__ == "__main__":
    # 打印环境信息
    logger.info(f"PyTorch version: {torch.__version__}")
    logger.info(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        logger.info(f"CUDA version: {torch.version.cuda}")
        logger.info(f"GPU count: {torch.cuda.device_count()}")
        logger.info(f"Current GPU: {torch.cuda.current_device()}")
        logger.info(f"GPU name: {torch.cuda.get_device_name(0)}")
    
    # 执行合并和保存
    merged_model, tokenizer = save_metrics_and_merge_model()
    
    # 测试模型
    test_merged_model(merged_model, tokenizer)

INFO:__main__:PyTorch version: 2.7.0+cu126
INFO:__main__:CUDA available: True
INFO:__main__:CUDA version: 12.6
INFO:__main__:GPU count: 1
INFO:__main__:Current GPU: 0
INFO:__main__:GPU name: B1.gpu.large
INFO:__main__:开始加载模型...
INFO:__main__:加载基础模型: /gemini/pretrain/DeepSeek-R1-Distill-Qwen-1.5B
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
INFO:__main__:应用LoRA配置...
INFO:__main__:加载最佳检查点: /gemini/pretrain2/deepseek-lora-best
INFO:__main__:合并模型权重...
INFO:__main__:保存合并后的模型到: /gemini/Arabic314891489/deepseek-merged
INFO:__main__:保存tokenizer...
INFO:__main__:所有操作完成！
INFO:__main__:
开始测试合并后的模型...
INFO:__main__:
测试提示: 请解释阿拉伯语中'الأولى'这个术语的含义。
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
INFO:__main__:模型回答: 请解释阿拉伯语中'الأولى'这个术语的含义。在什么情况下使用这个词？
</think>

在这个问题中，"أولى" 是阿拉伯语中一个常见的短语，意思是“最早”或“最先”。它通常用来表示相对于其他事物来说最早的，或者作为时间的起点。

例如：
- 如果我们说：“أولى الصراع”，意思是指最早发生的事件。
- 如果我们在谈论时间，我们会用“أولى”来指代过去最开始的时间点。

有时候，“أ

In [26]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载合并后的模型和tokenizer
model_path = "/gemini/Arabic314891489/deepseek-merged"
print(f"正在加载模型: {model_path}")

# 加载模型
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto"
)

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True
)

# 设置生成参数
generation_config = {
    "max_length": 800,
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.95,
    "repetition_penalty": 1.1,
    "num_return_sequences": 1
}

# 构建提示
prompts = ["请生成一段带有阿拉伯专业术语的阿拉伯语种文本","请生成一段带有阿拉伯专业术语的阿拉伯语种文本","请生成一段带有阿拉伯专业术语的阿拉伯语种文本","请生成一段带有阿拉伯专业术语的阿拉伯语种文本"]
print(f"提示: {prompts}")

# 对提示进行编码
inputs = tokenizer(prompts, return_tensors="pt").to(device)

# 生成文本
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        **generation_config
    )

# 解码并打印结果
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n生成的回复:")
print("-" * 50)
print(response)
print("-" * 50)

INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


正在加载模型: /gemini/Arabic314891489/deepseek-merged


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


提示: ['请生成一段带有阿拉伯专业术语的阿拉伯语种文本', '请生成一段带有阿拉伯专业术语的阿拉伯语种文本', '请生成一段带有阿拉伯专业术语的阿拉伯语种文本', '请生成一段带有阿拉伯专业术语的阿拉伯语种文本']

生成的回复:
--------------------------------------------------
请生成一段带有阿拉伯专业术语的阿拉伯语种文本，主题是关于概率论，包含“条件期望”和“联合分布”的概念。

请确保内容准确无误，符合学术规范，并且语言使用正确。
好的，我现在需要帮用户生成一段带有阿拉伯专业术语的阿拉伯语文本。主题是概率论，包含“条件期望”和“联合分布”。首先，我得确认用户的需求是什么。看起来他们可能在做相关的研究或教学，所以内容必须准确、符合学术规范。

接下来，我会考虑用户的深层需求。他们可能不仅想要一个段落，还希望这段文字能够清晰传达理论知识，帮助别人理解这些概念。因此，我需要用专业但易懂的语言来表达，确保术语准确，比如“ال condicion expected”（条件期望）和“ال joint distribution”（联合分布）。

然后，我要检查语法和用词是否正确，避免任何错误。阿拉伯语中的某些词汇可能会有特定的发音或者拼写，所以我需要确保没有遗漏任何细节。同时，保持句子结构流畅，逻辑清晰，让读者容易理解和掌握。

最后，我会通读整个文本，确保所有要点都涵盖了：定义条件期望和联合分布，以及它们之间的关系，并且说明如何从联合分布导出条件期望。这样用户就能得到一份完整且准确的材料了。
</think>

ال condicion expected (Conditional Expectation) و توزيع م joint (Joint Distribution) في احتمالات (Probability) تُعرف بان توزيع م joint يعتمد على قيم المتغيرات والجودة، بينما تُعرف بان توزيع م conditional يعتمد على قيم المتغيرات المذكورة. 

من الناحية التالية، تُعرف بان توزيع م joint ي encapsulates All possib

In [27]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 加载合并后的模型和tokenizer
model_path = "/gemini/Arabic314891489/deepseek-merged"
print(f"正在加载模型: {model_path}")

# 加载模型
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto"
)

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True
)

# 设置生成参数
generation_config = {
    "max_length": 800,
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.95,
    "repetition_penalty": 1.1,
    "num_return_sequences": 1
}

prompts = ["请生成一段带有阿拉伯专业术语的阿拉伯语种文本",
          "请生成一段带有阿拉伯专业术语的阿拉伯语种文本",
          "请生成一段带有阿拉伯专业术语的阿拉伯语种文本",
          "请生成一段带有阿拉伯专业术语的阿拉伯语种文本"]

for prompt_text in prompts:
    print(f"提示: {prompt_text}")
    # 对提示进行编码
    inputs = tokenizer(prompt_text, return_tensors="pt").to(device)

    # 生成文本
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            **generation_config
        )

    # 解码并打印结果
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("\n生成的回复:")
    print("-" * 50)
    print(response)
    print("-" * 50)

INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


正在加载模型: /gemini/Arabic314891489/deepseek-merged


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


提示: 请生成一段带有阿拉伯专业术语的阿拉伯语种文本


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



生成的回复:
--------------------------------------------------
请生成一段带有阿拉伯专业术语的阿拉伯语种文本，内容涉及数学或物理领域。
</think>

السلام عليكم！  
مرحباً في هذا اليوم، شكرًا لمساعدتك في تعلم有任何 شيء من التراث والتنبيه.  

للحصول على معلومات أكثر تفصيلاً في مجالات مثلMathematics und Physics، يُنصح بمتابعة النسخة الموثوقة وتعزيز الفهم بالتفاصيل التي ت用户提供ها.  

 best regards,  
[您的姓名]
--------------------------------------------------
提示: 请生成一段带有阿拉伯专业术语的阿拉伯语种文本


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



生成的回复:
--------------------------------------------------
请生成一段带有阿拉伯专业术语的阿拉伯语种文本，用于学术研究或教育用途。
</think>

"السياق البارد" (Bar扎i's Market)  
في عالم التحديات العينية، تُعتبر القيمة البارزة في حب العيد والضيوفة.  

الاستماعات الإخبارية (Exchange Rates) الباردية من الم markets البارдية يساعد على دراسة التحديات العينية وال Dinamic effects in exchange rates.
--------------------------------------------------
提示: 请生成一段带有阿拉伯专业术语的阿拉伯语种文本


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



生成的回复:
--------------------------------------------------
请生成一段带有阿拉伯专业术语的阿拉伯语种文本，主题是关于概率和统计学在人工智能中的应用。需要注意的是，要确保技术准确，且内容涵盖关键点如分布、模型评估等。

请从以下选项中选择合适的语言：
1. 中文
2. 阿拉伯语（阿拉伯语）
3. 英文

请给出具体的回答。
</think>

Sure! Here's a translated text:

**Probability and Statistics in Artificial Intelligence: A Comprehensive Overview**

In the realm of artificial intelligence, probability and statistics play pivotal roles in shaping the foundation for machine learning models and deep learning algorithms. These mathematical disciplines enable developers to design robust systems that can process vast amounts of data efficiently.

One key area where probability and statistics are applied is in probabilistic graphical models, which help in understanding complex relationships between variables within a system. Additionally, statistical methods like hypothesis testing are crucial in validating assumptions and ensuring the reliability of AI models.

Model evaluation is another critical aspect. Techniques such as

In [ ]:
# import os
# import shutil
# import time

# # 定义源目录和目标目录
# source_dir = "/home/mw/input"
# target_dir = "/home/mw/project"

# # 确保目标目录存在
# os.makedirs(target_dir, exist_ok=True)

# print(f"开始从 {source_dir} 复制文件到 {target_dir}...")

# # 获取源目录中所有内容
# all_items = []
# for root, dirs, files in os.walk(source_dir):
#     # 为每个目录创建相应的目标目录
#     for dir_name in dirs:
#         src_path = os.path.join(root, dir_name)
#         rel_path = os.path.relpath(src_path, source_dir)
#         dst_path = os.path.join(target_dir, rel_path)
#         os.makedirs(dst_path, exist_ok=True)
    
#     # 收集所有文件
#     for file_name in files:
#         src_file = os.path.join(root, file_name)
#         all_items.append(src_file)

# # 显示总文件数
# total_files = len(all_items)
# print(f"找到 {total_files} 个文件需要复制")

# # 复制文件并显示进度
# start_time = time.time()
# copied_size = 0
# copied_files = 0

# for src_file in all_items:
#     # 计算目标路径
#     rel_path = os.path.relpath(src_file, source_dir)
#     dst_file = os.path.join(target_dir, rel_path)
    
#     # 确保目标目录存在
#     os.makedirs(os.path.dirname(dst_file), exist_ok=True)
    
#     # 复制文件
#     try:
#         file_size = os.path.getsize(src_file)
#         shutil.copy2(src_file, dst_file)  # copy2保留元数据
#         copied_size += file_size
#         copied_files += 1
        
#         # 每5个文件显示一次进度
#         if copied_files % 5 == 0 or copied_files == total_files:
#             elapsed = time.time() - start_time
#             percent = copied_files / total_files * 100
#             size_mb = copied_size / (1024**2)
#             print(f"进度: {copied_files}/{total_files} ({percent:.1f}%) - 已复制 {size_mb:.2f} MB - 用时 {elapsed:.1f} 秒")
            
#     except Exception as e:
#         print(f"复制 {src_file} 时出错: {e}")

# # 显示完成信息
# elapsed_time = time.time() - start_time
# copied_gb = copied_size / (1024**3)  # 转换为GB

# print(f"\n复制完成!")
# print(f"总共复制了 {copied_files} 个文件，{copied_gb:.2f} GB")
# print(f"耗时: {elapsed_time:.2f} 秒，平均速度: {copied_gb/elapsed_time:.2f} GB/秒")

# # 验证复制是否成功
# source_file_count = sum([len(files) for _, _, files in os.walk(source_dir)])
# target_file_count = sum([len(files) for _, _, files in os.walk(target_dir)])

# print(f"\n验证结果:")
# print(f"源目录文件数: {source_file_count}")
# print(f"目标目录文件数: {target_file_count}")
# print(f"{'复制成功' if source_file_count == target_file_count else '复制可能不完整'}")

# 总结
本课程我们深入学习了大模型微调的全链条内容

数据集准备与清洗
编码
确定评估指标与训练方法
模型下载（部署）与训练
模型评估

数据集准备与清洗
我们通过opendatalab提供的原始数据，完成了“专业名词提取+IO数据对”的构造，IO数据对的格式满足"### Instruction:\n{prompt}\n\n### Response:\n{output}"
正确的数据格式才能带来有效的微调效果

编码
deepseek作为经典的因果语言，我们将数据集编码成符合因果语言训练的格式，该环节最终输出单向编码的数据，单向编码可以让模型在训练过程中学习训练集中的上下文语义关系

确定评估指标与训练方法
我们通过困惑度（perplexity）与 领域适应性评估（术语覆盖率、术语密度、响应质量）作为评估指标，这些指标也是LLM中常用的评估指标

模型下载（部署）与训练
我们指定了镜像环境，加快了模型下载速度，并且通过Lora微调方法在加快训练速度同时，保证训练质量

模型评估
我们对比了不同训练次数下的模型性能，“/gemini/Arabic314891489/deepseek-lora-best”权重目录下的模型性能最佳，也证明了评估指标与微调方法的有效性